## How To Take Time Traces to Determine Optimal Weights for Qubit Readout

### Prerequisites
This guide assumes you have a configured `DeviceSetup` as well as `Qubit` objects with assigned parameters. Please see these guides (add links) if you need to create your setup and qubits for the first time. However, you can also run this notebook "as is" using an emulated session. If you are just getting started with the LabOne Q Applications Library, please don't hesitate to reach out to us at info@zhinst.com.

### Background


In this how-to guide, you will measure the raw time traces of the readout signal coming back to the Quantum Analyzer unit of a SHFQC or SHFQA, before demodulation with the digital oscillator signal. This can be done here for every qubit in the sample for every possible combination of the states $\ket{g}$, $\ket{e}$ and $\ket{f}$. In a future version of the guide, the raw time traces will be used to determine the optimal integration kernel for qubit readout, achieving maximal distinguishability between different qubit states. 

To distinguish between $n$ different states, a total of $n(n-1)/2$ integration weights $w_{i, j}(t)$ is required, with $i, j \in\left\{0, 1, ..., n-1\right\}$ and $j>i$. At each time $t$, the optimal integration weights can be calculated as $w_{i, j}(t) = \overline{r_i(t) - r_j(t)}$, where $r_i$ denotes the time trace with the qubit in state $i$ and the horizontal bar represents the complex conjugate. Note that in general only $n-1$ different integration weights need to be explicitly measured since the remaining ones can be obtained from the pairwise difference of the measured weights.

### Imports

You'll start by importing the time traces experiment from `laboneq_applications`, as well as `laboneq.simple` and a demo QPU and device setup to run in emulation mode.

In [ ]:
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation
from laboneq.simple import *

from laboneq_applications.experiments import time_traces
from laboneq_applications.qpu_types.tunable_transmon.demo_qpus import demo_platform

### QPU and Device Setup

You'll generate two qubits with pre-defined parameters, as well as a `Device_Setup` consisting of a SHFQC+, HDAWG, and PQSC. If you already have your own `DeviceSetup` and qubits configured, you'll instead initialize the session using your setup.

In [ ]:
my_platform = demo_platform(2)

Then, you'll connect to the `Session`. Here we connect to an emulated one:

In [ ]:
session = Session(my_platform.setup)
session.connect(do_emulation=True)

### Running the weight optimization Workflow

You'll now make the experiment workflow and run:

In [ ]:
opts = time_traces.experiment_workflow.options()

In [ ]:
# our qubits live here in the demo setup:
qubits = [my_platform.qpu.qubits[0], my_platform.qpu.qubits[1]]
states = ["g", "e", "f"]

my_workflow = time_traces.experiment_workflow(
    session=session, qpu=my_platform.qpu, qubits=qubits, states=states, options=opts
)

my_results = my_workflow.run()

#### Output Simulation

You can also plot the simulated output:

In [ ]:
for compiled_exp in my_results.tasks["compile_experiment", :]:
    compiled_traces = compiled_exp.output
    plot_simulation(compiled_traces, start_time=0, length=10e-6)

Great! You've now run your Rabi experiment. Check out these other experiments to keep characterizing your qubits:

In [ ]:
# TODO: Add experiment links
# TODO: Add Analysis